In [59]:
import torch
import torchvision
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import numpy as np

In [60]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [61]:
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]
)

dataset = ImageFolder("train/", transform= transform)

In [62]:
seed = 8912
torch.manual_seed(seed)

In [63]:
val_size = 13500
train_size = len(dataset) - val_size

val_dataset, train_dataset = data.random_split(dataset, lengths=[val_size, train_size])

train_loader = data.DataLoader(train_dataset, batch_size=500, shuffle=True, pin_memory=True, num_workers=2)
val_loader = data.DataLoader(val_dataset, batch_size=500, pin_memory=True, num_workers=2)
print(len(val_dataset), len(train_dataset))

13500 74511


In [64]:
class Network(nn.Module):

    def __init__(self):
        super().__init__()
        self.loss = nn.CrossEntropyLoss()



    def train_step(self, input, labels):

        input = input.to(device=device, non_blocking=True)
        labels = labels.to(device=device, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(preds, labels)

        return loss

    def val_step(self, input, labels):

        input = input.to(device=device, non_blocking=True)
        labels = labels.to(device=device, non_blocking = True)

        preds = self.forward(input)
        loss = self.loss(input, labels)
        accuracy = self._accuracy(preds, labels)

        return {"loss":loss, "accuracy":accuracy}

    def val_epoch_end(self, preformance_measurement_data):

        accuracy = [x["accuracy"] for x in preformance_measurement_data]
        avg_accuracy = np.mean(accuracy)

        loss = [x["loss"] for x in preformance_measurement_data]
        avg_loss = np.mean(loss)

        return f'Epoch loss: {avg_loss}\nEpoch accuracy: {avg_accuracy}\n\n\n'


    def _accuracy(self, preds, labels):
        batch_size = len(preds)

        pred_indices = torch.argmax(preds, dim=1)
        return torch.tensor(torch.sum(pred_indices == labels).item() / batch_size)





In [65]:
nn = Network()